In [1]:
##imports

import random as rd
from pprint import pprint
import string

import nltk
from nltk.corpus import brown
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import sklearn 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef


In [2]:
##configuracao e funcoes auxiliares"""

##percentagem do conjunto de instancias que sera separado para validacao do classificador
test_set_rate=0.34


##numero de folds usado na estimativa dos hiperparametros de cada modelo
hyper_cv_folds = 3

##Permite que os resultados sejam reproduzidos exatamente usando a mesma semente 
##aleatoria. Defina False para habilitar resultados aleatorios
fixed_random_state = True

if fixed_random_state:
    rd.seed(1)
else:
    rd.seed()

#wrap funcion para o gerador de sementes aleatorias
def getRandIntState():
    if fixed_random_state:
        return 0
    else:
        return rd.getstate()

##pre processador     

stemmer = PorterStemmer()
translator = str.maketrans('', '', string.punctuation)

def analyzer1(words):
    ret = []
    
    for w in words:
        ##Remove stop words
        if w in stopwords.words("english"):
            continue
        ##Remove acentos, tranforma em minusculas
        w = w.translate(translator).lower()
        
        ##remove palavras vazias
        if len(w) == 0:
            continue
        
        ret.append(stemmer.stem(w))
    
    return ret


In [3]:
##Obtencao das instancias, dos labels, divisao treino/test

##cria uma distribuicao de frequencias para as categoria no corpo
freq_dist = {cat:len(brown.fileids(cat)) for cat in brown.categories()}

##Seleciona as duas categorias mais comuns
more_freq = sorted(freq_dist, key=lambda x: -freq_dist[x])[0:2]

##Extrai os ids das duas categorias mais comuns e cria os labels
raw_set = [(brown.words(text_id), more_freq[0]) for text_id in brown.fileids(more_freq[0])] + \
    [(brown.words(text_id), more_freq[1]) for text_id in brown.fileids(more_freq[1])]

#raw_set = [(brown.raw(text_id), more_freq[0]) for text_id in brown.fileids(more_freq[0])] + \
#    [(brown.raw(text_id), more_freq[1]) for text_id in brown.fileids(more_freq[1])]


print ("Total de {} instancias".format(len(raw_set)))    
print ("Sendo {} da classe \"{}\" e {} da classe \"{}\"".format(
                                                                len(brown.fileids(more_freq[0])),
                                                                more_freq[0], 
                                                                len(brown.fileids(more_freq[1])),
                                                                more_freq[1]))


##Conjunto das instancias
X = [r[0] for r in raw_set]

##Conjunto de labels
y = [r[1] for r in raw_set]

##Divide os conjuntos em teste e treino
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_set_rate, 
                                                    stratify=y, random_state=getRandIntState())


print ("Instancias: {} treino e {} teste".format(len(y_train), len(y_test)))    
    
    
##O fit e feito apenas no conjunto de treino para evitar information leak 
##para a etapa de validacao
tfidfBuilder = TfidfVectorizer(X_train, analyzer=analyzer1)

##Extrai os features para o conjunto de testes e de treino
X_train = tfidfBuilder.fit_transform(X_train)
X_test = tfidfBuilder.transform(X_test)

print ("Transformacao tdf-idf")
ex_raw = raw_set[0][0][:min(100, len(raw_set[0][0]))]
ex_tkn = list(tfidfBuilder.build_analyzer()(ex_raw))
ex_trans = tfidfBuilder.transform([ex_raw])

print ("\nExemplo de um texto de entrada:\n\n{}...".format(ex_raw))
print ("\nExemplo apos analyzer:\n\n{}...".format(ex_tkn))
print ("\nExemplo apos feature extraction:\n\n")
       
for vocab in ex_tkn[:15]:
    if vocab in tfidfBuilder.vocabulary_:
        fidf = ex_trans.getcol(tfidfBuilder.vocabulary_[vocab]).data
        print ("{}\t\t{}".format(vocab, fidf))

print("...")



Total de 155 instancias
Sendo 80 da classe "learned" e 75 da classe "belles_lettres"
Instancias: 102 treino e 53 teste
Transformacao tdf-idf

Exemplo de um texto de entrada:

['1', '.', 'Introduction', 'It', 'has', 'recently', 'become', 'practical', 'to', 'use', 'the', 'radio', 'emission', 'of', 'the', 'moon', 'and', 'planets', 'as', 'a', 'new', 'source', 'of', 'information', 'about', 'these', 'bodies', 'and', 'their', 'atmospheres', '.', 'The', 'results', 'of', 'present', 'observations', 'of', 'the', 'thermal', 'radio', 'emission', 'of', 'the', 'moon', 'are', 'consistent', 'with', 'the', 'very', 'low', 'thermal', 'conductivity', 'of', 'the', 'surface', 'layer', 'which', 'was', 'derived', 'from', 'the', 'variation', 'in', 'the', 'infrared', 'emission', 'during', 'eclipses', '(', 'e.g.', ',', 'Garstung', ',', '1958', ')', '.', 'When', 'sufficiently', 'accurate', 'and', 'complete', 'measurements', 'are', 'available', ',', 'it', 'will', 'be', 'possible', 'to', 'set', 'limits', 'on', 'the'

In [8]:
##Busca dos parametros

    
def selectBestParametersSet (clf, parameters):
    
    grid_search = GridSearchCV(
                                estimator=clf, 
                                param_grid=parameters, 
                                cv=StratifiedKFold(
                                                    n_splits=hyper_cv_folds, 
                                                    random_state=getRandIntState()),
                                verbose=1,
                                scoring="accuracy")

    grid_search.fit(X_train, y_train)
    results = grid_search.cv_results_
    params = results['params']
    
    for t in range(0, len(params)):
        print ("param set {}".format(t))
        pprint(params[t])

        mean = results['mean_test_score'][t]
        std = 2*results['std_test_score' ][t]
        rkn = results['rank_test_score'][t]
        
        print("scr:{}+/-{}. rnk{}".format(mean, std, rkn))

        
    best_clf = grid_search.best_estimator_
    best_params = grid_search.best_params_ 

    best_clf.fit(X_train, y_train)
    
    return {'clf':best_clf, 'params':best_params, 'pred':best_clf.predict(X_test)}

print("Tuning dos hiper-parametros")



print ("Decision Tree:")

tree = selectBestParametersSet(
                                DecisionTreeClassifier(random_state=getRandIntState()),
                               {'max_depth':[4, 8, 16, None], 
                                'criterion' : ['entropy', 'gini'],
                                'max_features' : [None, 'sqrt', 'log2']
                               })



print ("SVM-Classifier:")

svm = selectBestParametersSet(
                                SVC(random_state=getRandIntState()),
                                [{'kernel':['linear'], 'C':[0.0001, 0.1, 10, 100]} , 
                                 { 'kernel':['rbf'], 'C':[0.0001, 0.1, 10, 100], 
                                  'gamma' : [0.0001, 0.01, 0.1, 10, 100]}]
                               )

print ("Multilayer perceptron classifier:")

mlpc = selectBestParametersSet(
                                MLPClassifier(
                                    random_state=getRandIntState(), 
                                    max_iter=2000, 
                                    solver='lbfgs'),
                               {'hidden_layer_sizes':[(5, 5), (15,15), (10,5), (15, 15), 
                                                      (20, 50), (50,20)]})



print("fim do treino")

Tuning dos hiper-parametros
Decision Tree:
Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:    3.9s finished


param set 0
{'criterion': 'entropy', 'max_depth': 4, 'max_features': None}
scr:0.7156862745098039+/-0.031123277592579726. rnk1
param set 1
{'criterion': 'entropy', 'max_depth': 4, 'max_features': 'sqrt'}
scr:0.5980392156862745+/-0.2935738968274183. rnk20
param set 2
{'criterion': 'entropy', 'max_depth': 4, 'max_features': 'log2'}
scr:0.6568627450980392+/-0.14005296660786556. rnk6
param set 3
{'criterion': 'entropy', 'max_depth': 8, 'max_features': None}
scr:0.7156862745098039+/-0.031123277592579726. rnk1
param set 4
{'criterion': 'entropy', 'max_depth': 8, 'max_features': 'sqrt'}
scr:0.5882352941176471+/-0.2204646967537368. rnk22
param set 5
{'criterion': 'entropy', 'max_depth': 8, 'max_features': 'log2'}
scr:0.6274509803921569+/-0.09663883998904749. rnk14
param set 6
{'criterion': 'entropy', 'max_depth': 16, 'max_features': None}
scr:0.7156862745098039+/-0.031123277592579726. rnk1
param set 7
{'criterion': 'entropy', 'max_depth': 16, 'max_features': 'sqrt'}
scr:0.5882352941176471+/-0.

[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:   21.6s finished


param set 0
{'C': 0.0001, 'kernel': 'linear'}
scr:0.5196078431372549+/-0.013882833453779676. rnk6
param set 1
{'C': 0.1, 'kernel': 'linear'}
scr:0.5196078431372549+/-0.013882833453779676. rnk6
param set 2
{'C': 10, 'kernel': 'linear'}
scr:0.8529411764705882+/-0.09336983277773916. rnk1
param set 3
{'C': 100, 'kernel': 'linear'}
scr:0.8529411764705882+/-0.09336983277773916. rnk1
param set 4
{'C': 0.0001, 'gamma': 0.0001, 'kernel': 'rbf'}
scr:0.5196078431372549+/-0.013882833453779676. rnk6
param set 5
{'C': 0.0001, 'gamma': 0.01, 'kernel': 'rbf'}
scr:0.5196078431372549+/-0.013882833453779676. rnk6
param set 6
{'C': 0.0001, 'gamma': 0.1, 'kernel': 'rbf'}
scr:0.5196078431372549+/-0.013882833453779676. rnk6
param set 7
{'C': 0.0001, 'gamma': 10, 'kernel': 'rbf'}
scr:0.5196078431372549+/-0.013882833453779676. rnk6
param set 8
{'C': 0.0001, 'gamma': 100, 'kernel': 'rbf'}
scr:0.5196078431372549+/-0.013882833453779676. rnk6
param set 9
{'C': 0.1, 'gamma': 0.0001, 'kernel': 'rbf'}
scr:0.519607843

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:   44.0s finished


param set 0
{'hidden_layer_sizes': (5, 5)}
scr:0.803921568627451+/-0.1918489605320673. rnk2
param set 1
{'hidden_layer_sizes': (15, 15)}
scr:0.7745098039215687+/-0.1103998163701165. rnk4
param set 2
{'hidden_layer_sizes': (10, 5)}
scr:0.803921568627451+/-0.07919162979628427. rnk2
param set 3
{'hidden_layer_sizes': (15, 15)}
scr:0.7745098039215687+/-0.1103998163701165. rnk4
param set 4
{'hidden_layer_sizes': (20, 50)}
scr:0.8333333333333334+/-0.144985813350097. rnk1
param set 5
{'hidden_layer_sizes': (50, 20)}
scr:0.7647058823529411+/-0.07422512730342977. rnk6
fim do treino


In [9]:

def show_results(name, clf):
    print ("=================================================================================")
    print (name)
    print ("Melhores parametros => {}\n".format(clf['params']))
    print ("Matriz de confusao:")
    print (confusion_matrix(y_test, clf['pred']))
    print ("\nCoeficiente de correlacao de Matthews:")
    print (matthews_corrcoef(y_test, clf['pred']))
    print ("\nAcuracia e precisao:")
    print (classification_report(y_test, clf['pred']))
    print ("================================================================================")

show_results("Arvore de Decisao", tree)
show_results("SVM", svm)
show_results("MLPC", mlpc)
    


Arvore de Decisao
Melhores parametros => {'criterion': 'entropy', 'max_features': None, 'max_depth': 4}

Matriz de confusao:
[[21  5]
 [ 6 21]]

Coeficiente de correlacao de Matthews:
0.58547008547

Acuracia e precisao:
                precision    recall  f1-score   support

belles_lettres       0.78      0.81      0.79        26
       learned       0.81      0.78      0.79        27

   avg / total       0.79      0.79      0.79        53

SVM
Melhores parametros => {'kernel': 'linear', 'C': 10}

Matriz de confusao:
[[24  2]
 [ 6 21]]

Coeficiente de correlacao de Matthews:
0.706922820153

Acuracia e precisao:
                precision    recall  f1-score   support

belles_lettres       0.80      0.92      0.86        26
       learned       0.91      0.78      0.84        27

   avg / total       0.86      0.85      0.85        53

MLPC
Melhores parametros => {'hidden_layer_sizes': (20, 50)}

Matriz de confusao:
[[24  2]
 [ 6 21]]

Coeficiente de correlacao de Matthews:
0.706922820

__Desafio Data Science Intelivix - Avancado__


O corpus foi importado e a populacao de cada genero calculada, sendo os generos 'belles_lettres' e 'learned' os mais populosos. As entradas foram divididas em um conjunto de treino e um conjunto de testes. Cada entrada foi entao tokenizada utilizando-se a funcao analyzer1. Os tokens foram transformados em features utlizando-se a funcao TfidfVectorizer.

Tres classificadores foram escolhidos por representarem distintas metodologias de classificacao: 

1. Arvores de decisao, que a cada passo escolhe que caracteristica melhor divide as instancias baseado na minimizacao de algum criterio, isto e, qual o feature mais representativo para classificar uma instancia.

2. Maquinas de Vetores de suporte (SVM), que encontra quais as instancias mais representativas para se criar uma regiao de fronteira entre as duas classes das instancias.

3. Multilayer Perceptron (MLPC), que tenta criar uma representacao abstrata para as instancias investigando como combinacoes (somas) dos features podem decidir a classe que uma instacia pertence.

Cada um dos classificadores apresenta parametros que devem ser "tunados". Para cada classificador foram escolhidos os parametros mais representativos e um esquema 3-fold com estratificacao do conjunto de treino foi usado para definir o melhor conjunto de valores para esses parametros. A acuracia media (entre os folds) para cada conjunto de valores dos parametros foi utilizada como estimador para rankear e definir o melhor conjunto de valores para cada classificador.

Cada classificador tunado com os parametros otimos da etapa anterior foi treinado com todo o conjunto de treino e validado sob o conjunto de testes. Segue a analise dos resultados:

1. Arvores de decisao: Foi o classificador mais rapido a ser treinado, levando apenas 3.9s em todas as 72 dombinacoes de valores/folds. Em contrapartida, foi o que apresentou mais erros de classificacao, o que pode ser notado em pelo valor da acuracia e do coeficiente de correlacao de Matthews (CCW). Apresentou uma sensibilidade moderada a escolha dos parametros, apresentando uma diferenca de 0.127 entre sua melhor e pior acuracia. Alem disso, apresentou e uma alta variacao entre os folds (note o erro na estimativa da acuracia entre os folds), chagando a apresentar desvios da de 0.3 para alguns valores dos parametros. Altos valores do desvio padrao entre os folds sugerem uma dependencia do modelo com o conjunto de treino.

2. SVM: Foi o segundo classificador mais rapido, demandando 21.6s para ser treinado em todas as 72 possiveis combinacoes. Apresentou uma acuracia e um CCW superiores aos da arvore e decisao, estado empatado com o MLPC. Apresentou a maior sensibilidade aos parametros, com uma diferenca na acuracia de 0.334 entre o melhor e o pior caso. Entretanto, apresentou a menor variancia (maior estabilidade) dentre todos os classificadores entre os folds (independencia do conjunto de treino), sendo o maior desvio de 0.01. Este comprotamento sugere que uma invertigacao mais fina dos parametros pode levar a uma melhora desempenho deste classificador sem afetar sua estabilidade.


3. MLPC: Empatado com o SVM, foi o classificador mais lento, levando 44.0s para ser treinado nas 18 combinacoes possiveis. Apresentou a menor variacao com os parametros, sendo a diferenca entre o melhor e o pior caso da acuracia de 0.070. Entretanto, apresentou instabilidade entre as execucoes em diferentes foldes com os valores de paramentros fixados, mostrando desvios da ordem de 0.2. Isso sugere que poucas melhorias podem ser obtidas pelo ajuste dos parametros desse classificar. Seu tempo elevado de treinamento mostra-se um ponto negativo dessa tecnica.


Em resumo, a arvode de decisao mostrou-se um classificador rapido no treinamento porem, instavel e de acuracia menor. O MLPC mostra uma uma acuracia robusta para diferentes parametros, porem e lento e apresenta uma certa dependencia no conjunto de treino. O SVM foi o melhor classificador, com uma acuracia similar ao MLPC, um tempo de treinamento intermediario, menor dependencia com o conjunto de treino e possibilidades de melhorias. 

